In [1]:
from lusidtools.jupyter_tools import toggle_code

"""Setting up a blended benchmark

Demonstration of how to load a blended benchmark

Attributes
----------
Reference portfolios
Securitised portfolios
"""

toggle_code("Toggle Docstring")

In [2]:
# %load_ext lab_black
# %load_ext nb_black

# 1. Setup

In [3]:
# Import general purpose packages
import os
import json
import pandas as pd
import numpy as np
import datetime
import pytz
import warnings

warnings.filterwarnings("ignore", module="matplotlib*")

import matplotlib.pyplot as plt

from datetime import datetime, timedelta
from pandas import json_normalize
from flatten_json import flatten

# Import lusid specific packages
import lusid
import lusid.models as models
from lusidtools.cocoon.cocoon import load_from_data_frame

from lusid.utilities import ApiClientFactory
from lusidjam.refreshing_token import RefreshingToken

# Set display configuration
pd.set_option("display.max_columns", None)
pd.set_option("display.float_format", lambda x: "%.5f" % x)
pd.set_option("display.max_rows", 3500)
pd.set_option("max_colwidth", 20)

# Use line magic function to enable matplotlib to work interactively with iPython
%matplotlib inline

# Set style to fivethirtyeight to create clean and clear looking graphs
plt.style.use("fivethirtyeight")

# Define a dictionary containing default plotting configurations
params = {
    "legend.fontsize": "small",
    "figure.figsize": (12, 4.5),
    "axes.labelsize": "small",
    "axes.titlesize": "medium",
    "xtick.labelsize": "small",
    "ytick.labelsize": "small",
}

plt.rcParams.update(params)

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

api_status = pd.DataFrame(
    api_factory.build(lusid.api.ApplicationMetadataApi).get_lusid_versions().to_dict()
)

display(api_status)

,api_version,build_version,excel_version,links
0,v0,0.6.7104.0,0.5.2170,{'relation': 'Re...


In [4]:
# Define a scope to hold data

scope = "ukIBOR"
start_date = "2021-01-01"

In [5]:
# Define the APIs we use

reference_portfolios_api = api_factory.build(lusid.api.ReferencePortfolioApi)
instruments_api = api_factory.build(lusid.api.InstrumentsApi)

# Load instrument master

In [6]:
uk_stocks = pd.read_csv("data/benchmark/uk-stocks.csv")
uk_stocks.head(3)

,Ticker,Name,Sector,ISIN,SEDOL,Weighting
0,III LN,3i,Financial Services,GB00B1YW4409,B1YW440,0.04000
1,BKG LN,Berkeley Group H...,Household Goods ...,GB00B02L3W35,B02L3W3,0.04000
2,BATS LN,British American...,Tobacco,GB0002875804,287580,0.04000


In [7]:
# Load the instruments into LUSID

instrument_identifier_mapping = {
    "ClientInternal": "Ticker",
    "Isin": "ISIN",
    "Sedol": "SEDOL",
}

instrument_mapping_required = {"name": "Name"}

instrument_mapping_optional = {}

responses = load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=uk_stocks,
    mapping_required=instrument_mapping_required,
    mapping_optional=instrument_mapping_optional,
    file_type="instrument",
    identifier_mapping=instrument_identifier_mapping,
    property_columns=["Sector"],
)

In [8]:
uk_bonds = pd.read_csv("data/benchmark/uk-bonds.csv")
uk_bonds.head(3)

,ISIN,Name,Sector,Weighting
0,GB00BNNGP668,UKT 0 ⅜ 10/22/26,Government Bond,0.05000
1,GB00BNNGP775,UKT 0 ⅞ 01/31/46,Government Bond,0.10000
2,GB00BMBL1F74,UKT 0 ⅝ 10/22/50,Government Bond,0.15000


In [9]:
# Load the instruments into LUSID

instrument_identifier_mapping = {
    "ClientInternal": "ISIN",
    "Isin": "ISIN",
}

instrument_mapping_required = {"name": "Name"}

instrument_mapping_optional = {}

responses = load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=uk_bonds,
    mapping_required=instrument_mapping_required,
    mapping_optional=instrument_mapping_optional,
    file_type="instrument",
    identifier_mapping=instrument_identifier_mapping,
    property_columns=["Sector"],
)

# Create reference portfolios

In [10]:
uk_bond_index = "uKBondIndex"
uk_equity_index = "uKEquityIndex"
uk_blended_index = "ukBlendedIndex"

In [11]:
reference_portfolios = [uk_bond_index, uk_equity_index, uk_blended_index]

In [12]:
for portfolio in reference_portfolios:

    try:

        response = reference_portfolios_api.create_reference_portfolio(
            scope=scope,
            create_reference_portfolio_request=models.CreateReferencePortfolioRequest(
                display_name=portfolio, code=portfolio, created="2010-01-01"
            ),
        )

    except lusid.ApiException as e:
        print(json.loads(e.body)["title"])

Could not create a portfolio with id uKBondIndex because it already exists in scope ukIBOR.
Could not create a portfolio with id uKEquityIndex because it already exists in scope ukIBOR.
Could not create a portfolio with id ukBlendedIndex because it already exists in scope ukIBOR.


# Securitise the reference portfolios

In [13]:
for portfolio in reference_portfolios:

    response = instruments_api.upsert_instruments(
        request_body={
            "upsert_instrument": models.InstrumentDefinition(
                name=portfolio,
                identifiers={
                    "ClientInternal": models.InstrumentIdValue(value=portfolio)
                },
                look_through_portfolio_id=models.ResourceId(
                    scope=scope, code=portfolio
                ),
            )
        }
    )

# Load constituents for FI and EQ reference portfolios

In [14]:
for portfolio, df, instrument_id in [
    (uk_bond_index, uk_bonds, "ISIN"),
    (uk_equity_index, uk_stocks, "Ticker"),
]:

    # Initialise a list to hold our constituents
    constituents = []

    # Iterate over instrument unvierse to add each constituent to our list
    for _, row in df.iterrows():
        constituents.append(
            models.ReferencePortfolioConstituentRequest(
                instrument_identifiers={
                    "Instrument/default/ClientInternal": row[instrument_id]
                },
                weight=row["Weighting"],
                currency="GBP",
            )
        )

    # Create our request to add our constituents
    constituents_request = models.UpsertReferencePortfolioConstituentsRequest(
        effective_from=start_date,
        weight_type="Periodical",
        period_type="Quarterly",
        period_count=4,
        constituents=constituents,
    )

    # Call LUSID to upsert our constituents into our reference portfolio
    response = api_factory.build(
        lusid.api.ReferencePortfolioApi
    ).upsert_reference_portfolio_constituents(
        scope=scope,
        code=portfolio,
        upsert_reference_portfolio_constituents_request=constituents_request,
    )

    print(f"Constituents Upserted for {portfolio}")

Constituents Upserted for uKBondIndex
Constituents Upserted for uKEquityIndex


# Load constituents for blended benchmark

In [15]:
weightings = [(uk_bond_index, 0.6), (uk_equity_index, 0.4)]

In [16]:
# Initialise a list to hold our constituents
constituents = []

# Iterate over instrument unvierse to add each constituent to our list
for port, weighting in weightings:
    constituents.append(
        models.ReferencePortfolioConstituentRequest(
            instrument_identifiers={"Instrument/default/ClientInternal": port},
            weight=weighting,
            currency="GBP",
        )
    )

# Create our request to add our constituents
constituents_request = models.UpsertReferencePortfolioConstituentsRequest(
    effective_from=start_date,
    weight_type="Periodical",
    period_type="Quarterly",
    period_count=4,
    constituents=constituents,
)

# Call LUSID to upsert our constituents into our reference portfolio
response = api_factory.build(
    lusid.api.ReferencePortfolioApi
).upsert_reference_portfolio_constituents(
    scope=scope,
    code=uk_blended_index,
    upsert_reference_portfolio_constituents_request=constituents_request,
)

print(f"Constituents Upserted for {portfolio}")

Constituents Upserted for uKEquityIndex


# Get blended index

In [17]:
get_constituents = reference_portfolios_api.get_reference_portfolio_constituents(
    scope=scope, code=uk_blended_index
)

In [18]:
pd.DataFrame([flatten(item.to_dict()) for item in get_constituents.constituents])

,instrument_identifiers_Instrument/default/ClientInternal,instrument_uid,currency,properties,weight,floating_weight
0,uKBondIndex,LUID_HQ605RXU,GBP,{},0.60000,None
1,uKEquityIndex,LUID_VPAOW273,GBP,{},0.40000,None
